In [39]:
import pandas as pd
df = pd.read_csv('/content/drive/My Drive/Colab/sample_data/fake_or_real_news.csv')
df = df.drop(columns='title')
df = df.drop(columns='Unnamed: 0')


In [40]:
df.head()

,text,label
0,"Daniel Greenfield, a Shillman Journalism Fello...",FAKE
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,FAKE
2,U.S. Secretary of State John F. Kerry said Mon...,REAL
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",FAKE
4,It's primary day in New York and front-runners...,REAL


In [41]:
category = pd.get_dummies(df.label)
df_baru = pd.concat([df, category], axis=1)
df_baru = df_baru.drop(columns='label')
df_baru

,text,FAKE,REAL
0,"Daniel Greenfield, a Shillman Journalism Fello...",1,0
1,Google Pinterest Digg Linkedin Reddit Stumbleu...,1,0
2,U.S. Secretary of State John F. Kerry said Mon...,0,1
3,"— Kaydee King (@KaydeeKing) November 9, 2016 T...",1,0
4,It's primary day in New York and front-runners...,0,1
...,...,...,...
6330,The State Department told the Republican Natio...,0,1
6331,The ‘P’ in PBS Should Stand for ‘Plutocratic’ ...,1,0
6332,Anti-Trump Protesters Are Tools of the Oligar...,1,0
6333,"ADDIS ABABA, Ethiopia —President Obama convene...",0,1


In [42]:
news = df_baru['text'].values
label = df_baru[['FAKE', 'REAL']].values


In [43]:
from sklearn.model_selection import train_test_split

news_latih, news_test, label_latih, label_test = train_test_split(news, label, test_size=0.2)

In [44]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
 
tokenizer = Tokenizer(num_words=6335, oov_token='x')
tokenizer.fit_on_texts(news_latih) 
 
sekuens_latih = tokenizer.texts_to_sequences(news_latih)
sekuens_test = tokenizer.texts_to_sequences(news_test)
 
padded_latih = pad_sequences(sekuens_latih)
padded_test = pad_sequences(sekuens_test)


In [45]:
import tensorflow as tf
model = tf.keras.Sequential([
    tf.keras.layers.Embedding(input_dim=6335, output_dim=16),
    tf.keras.layers.LSTM(64),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(2, activation='sigmoid')
])


In [46]:
model.compile(loss='binary_crossentropy',
              optimizer=tf.optimizers.Adam(),
              metrics=['accuracy'])

In [47]:
class myCallback(tf.keras.callbacks.Callback):
  def on_epoch_end(self, epoch, logs={}):
    if(logs.get('val_accuracy')>0.91):
      print("\nAkurasi telah mencapai >91%!")
      self.model.stop_training = True
callbacks = myCallback()

In [48]:
num_epochs = 20
history = model.fit(padded_latih, label_latih, epochs=num_epochs, 
          validation_data=(padded_test, label_test), verbose=1,
          callbacks=[callbacks])

Epoch 1/20
159/159 [==============================] - 316s 2s/step - loss: 0.5986 - accuracy: 0.6902 - val_loss: 0.4027 - val_accuracy: 0.8122
Epoch 2/20
159/159 [==============================] - 316s 2s/step - loss: 0.2823 - accuracy: 0.8860 - val_loss: 0.3081 - val_accuracy: 0.8674
Epoch 3/20
159/159 [==============================] - 318s 2s/step - loss: 0.1495 - accuracy: 0.9475 - val_loss: 0.3497 - val_accuracy: 0.8579
Epoch 4/20
159/159 [==============================] - 319s 2s/step - loss: 0.0855 - accuracy: 0.9732 - val_loss: 0.3842 - val_accuracy: 0.8674
Epoch 5/20
159/159 [==============================] - 319s 2s/step - loss: 0.0578 - accuracy: 0.9805 - val_loss: 0.4374 - val_accuracy: 0.8713
Epoch 6/20
159/159 [==============================] - 318s 2s/step - loss: 0.0181 - accuracy: 0.9955 - val_loss: 0.6387 - val_accuracy: 0.8721
Epoch 7/20
159/159 [==============================] - 317s 2s/step - loss: 0.0313 - accuracy: 0.9899 - val_loss: 0.6567 - val_accuracy: 0.8564